In [1]:
!pip install -q efficientnet
!pip install keras-rectified-adam
!pip install keras-lookahead
import os
import re
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
import efficientnet.tfkeras as efn
import dill
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
from keras_radam import RAdam
from keras_lookahead import Lookahead
from tensorflow.keras.callbacks import ModelCheckpoint


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.20.0-py3-none-any.whl size=8271 sha256=36f0ded84fddc6a354cf1fe4ffd65d1465164da3cce2de23e678d995bc135ade
  Stored in directory: /root/.cache/pip/wheels/5d/0c/92/c76f52204fb345e4495a09929ffa813a0d790e971f6064a6c9
Successfully built keras-rectified-adam

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
  Created wheel for keras-lookahead: filename=keras_lookahead-0.9.0-py3-none-any.whl size=6402 sha256=067961d3c7cb3891e7af9f48bf4c3cfca85ab365a841a27299e432b787db338d
  Stored in directory: /root/.cache/pip/wheels/51/6d/2c/fc2fe9362e1159ba178f3bcc96feb5d9d392ed28d20db25728
Successfully built keras-lookahead

[notice] A new release of pip is available: 23.0.1 -> 23.

D1101 06:54:34.929979200      15 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D1101 06:54:34.930012491      15 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D1101 06:54:34.930016641      15 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D1101 06:54:34.930019661      15 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D1101 06:54:34.930022661      15 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D1101 06:54:34.930025639      15 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D1101 06:54:34.930033074      15 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D1101 06:54:34.

In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


REPLICAS:  8


In [3]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_PATH = KaggleDatasets().get_gcs_path('melanoma-256x256')

# Configuration
EPOCHS = 25    
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
AUG_BATCH = BATCH_SIZE
IMAGE_SIZE = [256, 256]
# Seed
SEED = 123
# Learning rate
LR = 0.00001
# cutmix prob
cutmix_rate = 0.30

# training filenames directory
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train*.tfrec')
# test filenames directory
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test*.tfrec')


# submission file
SUB = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')

get_gcs_path is not required on TPU VMs which can directly use Kaggle datasets, using path: /kaggle/input/melanoma-256x256


In [4]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

In [5]:
def transform(image, label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    tmp = random.uniform(0, 1)
    if 0 < tmp <= 0.1:
        rot = 15.0 * tf.random.normal([1],dtype='float32')
    elif 0.1 < tmp <= 0.2:
        rot = 30.0 * tf.random.normal([1],dtype='float32')
    elif 0.2 < tmp <= 0.3:
        rot = 45.0 * tf.random.normal([1],dtype='float32')
    elif 0.3 < tmp <= 0.4:
        rot = 60.0 * tf.random.normal([1],dtype='float32')
    elif 0.4 < tmp <= 0.5:
        rot = 75.0 * tf.random.normal([1],dtype='float32')
    elif 0.5 < tmp <= 0.6:
        rot = 90.0 * tf.random.normal([1],dtype='float32')
    elif 0.6 < tmp <= 0.7:
        rot = 110.0 * tf.random.normal([1],dtype='float32')
    elif 0.7 < tmp <= 0.8:
        rot = 130.0 * tf.random.normal([1],dtype='float32')
    elif 0.8 < tmp <= 0.9:
        rot = 150.0 * tf.random.normal([1],dtype='float32')
    elif 0.9 < tmp <= 1.0:
        rot = 180.0 * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image['inp1'],tf.transpose(idx3))
        
    return {'inp1': tf.reshape(d,[DIM,DIM,3]), 'inp2': image['inp2']}, label


In [6]:
# function to apply cutmix augmentation
def cutmix(image, label):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    
    DIM = IMAGE_SIZE[0]    
    imgs = []; labs = []
    
    for j in range(BATCH_SIZE):
        
        #random_uniform( shape, minval=0, maxval=None)        
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast(tf.random.uniform([], 0, 1) <= cutmix_rate, tf.int32)
        
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast(tf.random.uniform([], 0, BATCH_SIZE), tf.int32)
        
        # CHOOSE RANDOM LOCATION
        x = tf.cast(tf.random.uniform([], 0, DIM), tf.int32)
        y = tf.cast(tf.random.uniform([], 0, DIM), tf.int32)
        
        # Beta(1, 1)
        b = tf.random.uniform([], 0, 1) # this is beta dist with alpha=1.0
        

        WIDTH = tf.cast(DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        
        # MAKE CUTMIX IMAGE
        one = image['inp1'][j,ya:yb,0:xa,:]
        two = image['inp1'][k,ya:yb,xa:xb,:]
        three = image['inp1'][j,ya:yb,xb:DIM,:]        
        #ya:yb
        middle = tf.concat([one,two,three],axis=1)

        img = tf.concat([image['inp1'][j,0:ya,:,:],middle,image['inp1'][j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        lab1 = label[j,]
        lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)

    image2 = tf.reshape(tf.stack(imgs),(BATCH_SIZE,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(BATCH_SIZE, 1))
    return {'inp1': image2, 'inp2': image['inp2']}, label2



In [7]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [8]:
# function to decode our images (normalize and reshape)
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    # convert image to floats in [0, 1] range
    image = tf.cast(image, tf.float32) / 255.0 
    # explicit size needed for TPU
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image


In [9]:
# this function parse our images and also get the target variable
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        "target": tf.io.FixedLenFeature([], tf.int64),
        # meta features
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([],   tf.int64)
        
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['target'], tf.float32)
    # meta features
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    # returns a dataset of (image, label, data)
    return image, label, data



In [10]:
# this function parse our image and also get our image_name (id) to perform predictions
def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        "image_name": tf.io.FixedLenFeature([], tf.string),
        # meta features
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    image_name = example['image_name']
    # meta features
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    # returns a dataset of (image, key, data)
    return image, image_name, data


In [11]:
def load_dataset(filenames, labeled = True, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        # disable order, increase speed
        ignore_order.experimental_deterministic = False 
        
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # use data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order)
    # returns a dataset of (image, label) pairs if labeled = True or (image, id) pair if labeld = False
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls = AUTO) 
    return dataset



In [12]:
# function for training and validation dataset
def setup_input1(image, label, data):
    
    # get anatom site general challenge vectors
    anatom = [tf.cast(data['anatom_site_general_challenge'][i], dtype = tf.float32) for i in range(7)]
    
    tab_data = [tf.cast(data[tfeat], dtype = tf.float32) for tfeat in ['age_approx', 'sex']]
    
    tabular = tf.stack(tab_data + anatom)
    
    return {'inp1': image, 'inp2':  tabular}, label

In [13]:
# function for the test set
def setup_input2(image, image_name, data):
    
    # get anatom site general challenge vectors
    anatom = [tf.cast(data['anatom_site_general_challenge'][i], dtype = tf.float32) for i in range(7)]
    
    tab_data = [tf.cast(data[tfeat], dtype = tf.float32) for tfeat in ['age_approx', 'sex']]
    
    tabular = tf.stack(tab_data + anatom)
    
    return {'inp1': image, 'inp2':  tabular}, image_name

In [14]:
# function for the validation (image name)
def setup_input3(image, image_name, target, data):
    
    # get anatom site general challenge vectors
    anatom = [tf.cast(data['anatom_site_general_challenge'][i], dtype = tf.float32) for i in range(7)]
    
    tab_data = [tf.cast(data[tfeat], dtype = tf.float32) for tfeat in ['age_approx', 'sex']]
    
    tabular = tf.stack(tab_data + anatom)
    
    return {'inp1': image, 'inp2':  tabular}, image_name, target

In [15]:
def data_augment(data, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement 
    # in the next function (below), this happens essentially for free on TPU. 
    # Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    data['inp1'] = tf.image.random_flip_left_right(data['inp1'])
    data['inp1'] = tf.image.random_flip_up_down(data['inp1'])
    data['inp1'] = tf.image.random_hue(data['inp1'], 0.01)
    data['inp1'] = tf.image.random_saturation(data['inp1'], 0.7, 1.3)
    data['inp1'] = tf.image.random_contrast(data['inp1'], 0.8, 1.2)
    data['inp1'] = tf.image.random_brightness(data['inp1'], 0.1)
    #data['inp1'] = tf.image.random_crop(data['inp1'], 0.1)
    #data['inp1'] = tf.image.central_crop(data['inp1'], 0.1)
    
    #data['inp1'] = data_augmentation(data['inp1'], label['inp1'])
    return data, label

In [16]:
def data_augment_spatial(image, label):
    p_spatial = tf.random.uniform([1], minval=0, maxval=1, dtype='float32')
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if p_spatial > .75:
        image = tf.image.transpose(image)

    return image, label

def data_augment_rotate(image, label):
    p_rotate = tf.random.uniform([1], minval=0, maxval=1, dtype='float32')
    
    if p_rotate > .66:
        image = tf.image.rot90(image, k=3) # rotate 270º
    elif p_rotate > .33:
        image = tf.image.rot90(image, k=2) # rotate 180º
    else:
        image = tf.image.rot90(image, k=1) # rotate 90º

    return image, label

def data_augment_crop(image, label):
    p_crop = tf.random.uniform([1], minval=0, maxval=1, dtype='float32')
    
    if p_crop > .8:
        image = tf.image.random_crop(image, size=[int(config['HEIGHT']*.7), int(config['WIDTH']*.7), config['CHANNELS']])
    elif p_crop > .6:
        image = tf.image.random_crop(image, size=[int(config['HEIGHT']*.8), int(config['WIDTH']*.8), config['CHANNELS']])
    elif p_crop > .4:
        image = tf.image.random_crop(image, size=[int(config['HEIGHT']*.9), int(config['WIDTH']*.9), config['CHANNELS']])
    elif p_crop > .2:
        image = tf.image.central_crop(image, central_fraction=.8)
    else:
        image = tf.image.central_crop(image, central_fraction=.7)
    
    image = tf.image.resize(image, size=[config['HEIGHT'], config['WIDTH']])

    return image, label

def data_augment_rotation(image, label, max_angle=45.):
    image = transform_rotation(image, config['HEIGHT'], max_angle)
        
    return image, label

def data_augment_shift(image, label):
    image = transform_shift(image, config['HEIGHT'], 50., 50.)
    return image, label

def data_augment_shear(image, label):
    image = transform_shear(image, config['HEIGHT'], 25.)
    return image, label

def data_augment_hue(image, label):
    image = tf.image.random_hue(image, 0.02)
    return image, label

def data_augment_saturation(image, label):
    image = tf.image.random_saturation(image, 0.8, 1.2)
    return image, label

def data_augment_contrast(image, label):
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image, label

def data_augment_brightness(image, label):
    image = tf.image.random_brightness(image, 0.1)
    return image, label

def data_augment_cutout(image, label):
    p_cutout = tf.random.uniform([1], minval=0, maxval=1, dtype='float32')
    
    if p_cutout > .9: # 3 cut outs
        image = random_cutout(image, config['HEIGHT'], config['WIDTH'], min_mask_size=(10, 10), max_mask_size=(80, 80), k=3)
    elif p_cutout > .75: # 2 cut outs
        image = random_cutout(image, config['HEIGHT'], config['WIDTH'], min_mask_size=(10, 10), max_mask_size=(80, 80), k=2)
    else: # 1 cut out
        image = random_cutout(image, config['HEIGHT'], config['WIDTH'], min_mask_size=(10, 10), max_mask_size=(80, 80), k=1)
        
    return image, label

def data_augmentation(image, label):
    image, label = data_augment_spatial(image, label)
    image, label = data_augment_rotate(image, label)
    image, label = data_augment_crop(image, label)
    image, label = data_augment_rotation(image, label)
    image, label = data_augment_shift(image, label)
    image, label = data_augment_shear(image, label)
    image, label = data_augment_hue(image, label)
    image, label = data_augment_saturation(image, label)
    image, label = data_augment_contrast(image, label)
    image, label = data_augment_brightness(image, label)
    
    return image, label

In [17]:
def get_training_dataset(filenames, labeled = True, ordered = False):
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(setup_input1, num_parallel_calls = AUTO)
    dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    dataset = dataset.map(transform, num_parallel_calls = AUTO)
    # the training dataset must repeat for several epochs
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    
    dataset = dataset.prefetch(AUTO)
    return dataset

In [18]:
def get_validation_dataset(filenames, labeled = True, ordered = True):
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(setup_input1, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    # using gpu, not enought memory to use cache
    # dataset = dataset.cache()
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

In [19]:
def get_test_dataset(filenames, labeled = False, ordered = True):
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(setup_input2, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

In [20]:
# function to count how many photos we have in
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

# this function parse our images and also get the target variable
def read_tfrecord_full(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), 
        "image_name": tf.io.FixedLenFeature([], tf.string), 
        "target": tf.io.FixedLenFeature([], tf.int64), 
        # meta features
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    image_name = example['image_name']
    target = tf.cast(example['target'], tf.float32)
    # meta features
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    return image, image_name, target, data

In [21]:
def load_dataset_full(filenames):        
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # returns a dataset of (image_name, target)
    dataset = dataset.map(read_tfrecord_full, num_parallel_calls = AUTO) 
    return dataset

In [22]:
def get_data_full(filenames):
    dataset = load_dataset_full(filenames)
    dataset = dataset.map(setup_input3, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

In [23]:
NUM_TRAINING_IMAGES = int(count_data_items(TRAINING_FILENAMES) * 0.8)
# use validation data for training
NUM_VALIDATION_IMAGES = int(count_data_items(TRAINING_FILENAMES) * 0.2)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

Dataset: 26153 training images, 6538 validation images, 10982 unlabeled test images


In [24]:
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

In [25]:
def get_model():
    
    
    with strategy.scope():
        inp1 = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        inp2 = tf.keras.layers.Input(shape = (9), name = 'inp2')
        efnetb0 = efn.EfficientNetB6(weights = 'imagenet', include_top = False)
        x = efnetb0(inp1)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x1 = tf.keras.layers.Dense(50)(inp2)
        x1 = tf.keras.layers.BatchNormalization()(x1)
        x1 = tf.keras.layers.Activation('relu')(x1)
        concat = tf.keras.layers.concatenate([x, x1])
        concat = tf.keras.layers.Dense(384, activation = 'relu')(concat)
        concat = tf.keras.layers.BatchNormalization()(concat)
        concat = tf.keras.layers.Dropout(0.2)(concat)
        concat = tf.keras.layers.Dense(182, activation = 'relu')(concat)
        concat = tf.keras.layers.BatchNormalization()(concat)
        concat = tf.keras.layers.Dropout(0.2)(concat)
        output = tf.keras.layers.Dense(1, activation = 'sigmoid')(concat)

        model = tf.keras.models.Model(inputs = [inp1, inp2], outputs = [output])

        #opt = tf.keras.optimizers.Adam(learning_rate = LR)
        # opt = tfa.optimizers.SWA(opt)
        #opt = Lookahead(RAdam(lr=0.001))
        #opt = RAdam(total_steps=5000, warmup_proportion=0.1,learning_rate = 1e-4)
        
        opt = tfa.optimizers.RectifiedAdam(lr=1e-3,total_steps=10000, warmup_proportion=0.1,min_lr=1e-5,)
        opt = tfa.optimizers.Lookahead(opt, sync_period=6, slow_step_size=0.5)
        
        model.compile(
            optimizer = opt,
            loss = [binary_focal_loss(gamma = 2.0, alpha = 0.80)],
            metrics = [tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()]
        )
        
        return model

In [26]:
print("Model Summary")
print(get_model().summary())

Model Summary
165527152/165527152 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inp2 (InputLayer)              [(None, 9)]          0           []                               
                                                                                                  
 inp1 (InputLayer)              [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 dense (Dense)                  (None, 50)           500         ['inp2[0][0]']                   
                                                                                                  
 efficientnet

/usr/local/lib/python3.8/site-packages/tensorflow_addons/optimizers/rectified_adam.py:121: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Trainable params: 41,712,009
Non-trainable params: 225,664
__________________________________________________________________________________________________
None


In [27]:
file_path="/kaggle/working/effnet4_weights.best.hdf5"

checkpoint = ModelCheckpoint(file_path, monitor='auc', verbose=1, save_best_only=True, mode='max')

In [ ]:
def train_and_predict(SUB, folds = 2):
    
    models = []
    oof_image_name = []
    oof_target = []
    oof_prediction = []
    
    # seed everything
    seed_everything(SEED)

    kfold = KFold(folds, shuffle = True, random_state = SEED)
    #kfold = StratifiedKFold(folds, shuffle=True, random_state = SEED)
    #for fold, (trn_ind, val_ind) in enumerate(kfold.split(TRAINING_FILENAMES,trn_ind)):
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(TRAINING_FILENAMES)):
        print('\n')
        print('-'*50)
        print(f'Training fold {fold + 1}')
        train_dataset = get_training_dataset([TRAINING_FILENAMES[x] for x in trn_ind], labeled = True, ordered = False)
        val_dataset = get_validation_dataset([TRAINING_FILENAMES[x] for x in val_ind], labeled = True, ordered = True)
        K.clear_session()
        model = get_model()
       
        # using early stopping using val loss
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_auc', mode = 'max', patience = 5, 
                                                      verbose = 1, min_delta = 0.0001, restore_best_weights = True)
        # lr scheduler
        cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_auc', factor = 0.4, patience = 2, verbose = 1, min_delta = 0.0001, mode = 'max')
        history = model.fit(train_dataset, 
                            steps_per_epoch = STEPS_PER_EPOCH,
                            epochs = EPOCHS,
                            callbacks = [checkpoint, early_stopping, cb_lr_schedule],
                            validation_data = val_dataset,
                            verbose = 2)
        models.append(model)
        
        # want to predict the validation set and save them for stacking
        number_of_files = count_data_items([TRAINING_FILENAMES[x] for x in val_ind])
        dataset = get_data_full([TRAINING_FILENAMES[x] for x in val_ind])
        # get the image name
        image_name = dataset.map(lambda image, image_name, target: image_name).unbatch()
        image_name = next(iter(image_name.batch(number_of_files))).numpy().astype('U')
        # get the real target
        target = dataset.map(lambda image, image_name, target: target).unbatch()
        target = next(iter(target.batch(number_of_files))).numpy()
        # predict the validation set
        image = dataset.map(lambda image, image_name, target: image)
        probabilities = model.predict(image)
        oof_image_name.extend(list(image_name))
        oof_target.extend(list(target))
        oof_prediction.extend(list(np.concatenate(probabilities)))
    
    print('\n')
    print('-'*50)
    # save predictions
    oof_df = pd.DataFrame({'image_name': oof_image_name, 'target': oof_target, 'predictions': oof_prediction})
    oof_df.to_csv('Efficient_B6.csv', index = False)
        
    # since we are splitting the dataset and iterating separately on images and ids, order matters.
    test_ds = get_test_dataset(TEST_FILENAMES, labeled = False, ordered = True)
    test_images_ds = test_ds.map(lambda image, image_name: image)
    
    print('Computing predictions...')
    probabilities = np.average([np.concatenate(models[i].predict(test_images_ds)) for i in range(folds)], axis = 0)
    print('Generating submission.csv file...')
    test_ids_ds = test_ds.map(lambda image, image_name: image_name).unbatch()
    # all in one batch
    test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
    pred_df = pd.DataFrame({'image_name': test_ids, 'target': probabilities})
    SUB.drop('target', inplace = True, axis = 1)
    SUB = SUB.merge(pred_df, on = 'image_name')
    SUB.to_csv('sub_Efficient_B6.csv', index = False)
    
    return oof_target, oof_prediction
    
oof_target, oof_prediction = train_and_predict(SUB)



--------------------------------------------------
Training fold 1
Epoch 1/25


2023-11-01 07:06:02.984930: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-11-01 07:06:07.978545: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-11-01 07:12:35.885001: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-11-01 07:12:36.466374: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.



Epoch 1: auc improved from -inf to 0.50151, saving model to /kaggle/working/effnet4_weights.best.hdf5
408/408 - 585s - loss: 0.7699 - binary_accuracy: 0.5232 - auc: 0.5015 - val_loss: 0.2997 - val_binary_accuracy: 0.6368 - val_auc: 0.5207 - lr: 0.0010 - 585s/epoch - 1s/step
Epoch 2/25

Epoch 2: auc improved from 0.50151 to 0.54676, saving model to /kaggle/working/effnet4_weights.best.hdf5
408/408 - 96s - loss: 0.3457 - binary_accuracy: 0.7040 - auc: 0.5468 - val_loss: 0.1238 - val_binary_accuracy: 0.9621 - val_auc: 0.6579 - lr: 0.0010 - 96s/epoch - 234ms/step
Epoch 3/25

Epoch 3: auc improved from 0.54676 to 0.60583, saving model to /kaggle/working/effnet4_weights.best.hdf5
408/408 - 97s - loss: 0.1701 - binary_accuracy: 0.8961 - auc: 0.6058 - val_loss: 0.1116 - val_binary_accuracy: 0.9813 - val_auc: 0.6685 - lr: 0.0010 - 97s/epoch - 237ms/step
Epoch 4/25

Epoch 4: auc improved from 0.60583 to 0.66421, saving model to /kaggle/working/effnet4_weights.best.hdf5
408/408 - 97s - loss: 0.1

2023-11-01 07:50:26.772978: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-11-01 07:50:27.243372: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


102/102 [==============================] - 26s 54ms/step


--------------------------------------------------
Training fold 2
Epoch 1/25


...


Epoch 1: auc did not improve from 0.90444
408/408 - 290s - loss: 0.7636 - binary_accuracy: 0.5255 - auc: 0.5501 - val_loss: 0.2972 - val_binary_accuracy: 0.6580 - val_auc: 0.5628 - lr: 0.0010 - 290s/epoch - 710ms/step
Epoch 2/25

Epoch 2: auc did not improve from 0.90444
408/408 - 91s - loss: 0.3507 - binary_accuracy: 0.6986 - auc: 0.5849 - val_loss: 0.1711 - val_binary_accuracy: 0.8694 - val_auc: 0.6335 - lr: 0.0010 - 91s/epoch - 223ms/step
Epoch 3/25

Epoch 3: auc did not improve from 0.90444
408/408 - 91s - loss: 0.1809 - binary_accuracy: 0.8926 - auc: 0.5939 - val_loss: 0.1030 - val_binary_accuracy: 0.9798 - val_auc: 0.7589 - lr: 0.0010 - 91s/epoch - 224ms/step
Epoch 4/25

Epoch 4: auc did not improve from 0.90444
408/408 - 91s - loss: 0.1467 - binary_accuracy: 0.9395 - auc: 0.6353 - val_loss: 0.0947 - val_binary_accuracy: 0.9792 - val_auc: 0.8056 - lr: 0.0010 - 91s/epoch - 223ms/step
Epoch 5/25

Epoch 5: auc did not improve from 0.90444
408/408 - 91s - loss: 0.1296 - binary_accur

2023-11-01 09:40:50.040544: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-11-01 09:40:50.511519: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


103/103 [==============================] - 12s 56ms/step


--------------------------------------------------
Training fold 5
Epoch 1/25


2023-11-01 09:44:16.386172: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-11-01 09:44:21.247960: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-11-01 09:46:02.039399: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-11-01 09:46:02.679986: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.



Epoch 1: auc did not improve from 0.90444
408/408 - 278s - loss: 0.7573 - binary_accuracy: 0.5209 - auc: 0.5355 - val_loss: 0.3424 - val_binary_accuracy: 0.5479 - val_auc: 0.5489 - lr: 0.0010 - 278s/epoch - 681ms/step
Epoch 2/25

Epoch 2: auc did not improve from 0.90444
408/408 - 90s - loss: 0.3475 - binary_accuracy: 0.7002 - auc: 0.5685 - val_loss: 0.1329 - val_binary_accuracy: 0.9442 - val_auc: 0.6710 - lr: 0.0010 - 90s/epoch - 221ms/step
Epoch 3/25

Epoch 3: auc did not improve from 0.90444
408/408 - 91s - loss: 0.1762 - binary_accuracy: 0.8988 - auc: 0.5857 - val_loss: 0.1003 - val_binary_accuracy: 0.9821 - val_auc: 0.7787 - lr: 0.0010 - 91s/epoch - 223ms/step
Epoch 4/25

Epoch 4: auc did not improve from 0.90444
408/408 - 88s - loss: 0.1453 - binary_accuracy: 0.9460 - auc: 0.6429 - val_loss: 0.0975 - val_binary_accuracy: 0.9783 - val_auc: 0.7724 - lr: 0.0010 - 88s/epoch - 216ms/step
Epoch 5/25

Epoch 5: auc did not improve from 0.90444
408/408 - 90s - loss: 0.1345 - binary_accur

In [ ]:
# calculate our out of folds roc auc score
roc_auc = metrics.roc_auc_score(oof_target, oof_prediction)
print('Our out of folds roc auc score is: ', roc_auc)

In [ ]:
plt.plot(history.history['auc'])
plt.title('model auc')
plt.ylabel('auc')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.savefig('auc.jpg')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.savefig('loss.jpg')
plt.show()